# The BatAnalysis Survey Data Analysis Code

### This notebook will go through how the BatAnalysis code can be used to analyze BAT survey data.

This notebook can be run interactively on your local machine if you:
1. Clone the git repository with 
```
git clone https://github.com/parsotat/BatAnalysis.git
```
2. After, the notebook can be accessed by navigating to the `notebook/` directory and running 
```
jupyter notebook
```
in the command line.

This notebook only covers how to use the code to manipulate survey data. It does not go into the details of what some of the in depth analysis details are. To understand that, we urge the user to read the Bat User Manual which can be accessed at [https://swift.gsfc.nasa.gov/analysis/bat_swguide_v6_3.pdf](https://swift.gsfc.nasa.gov/analysis/bat_swguide_v6_3.pdf). This guide provides a detailed explaination for the steps that are taken to analyze BAT data. 

Since the BatAnalysis code still calls heasoft tools, knowing the various bat related heasoft resources are important as well so one has an idea of what parameters and value pairs can be passed to these scripts. Typically googling `heasoft xxxx` where xxxx is whatever script is of interest should bring up a webpage describing the code. Alternatively searching for the script here [https://heasarc.gsfc.nasa.gov/ftools/caldb/help/](https://heasarc.gsfc.nasa.gov/ftools/caldb/help/) will also allow one to get to that same information.

### 1a. First we need to install the package which can be done using *pip*

If this is a developer version of the code, navigate to the BatAnalysis/ directory and run:
```
pip install -e .
```
otherwise you can simply run:
```
pip install BatAnalysis
```

Now, the BatAnalysis code can be accessed anywhere on your computer and improrted like any other python package.

### 1a. Then we need to download the BAT pattern noise maps from [https://zenodo.org/record/7595904#.Y9q7pS-B3T8](https://zenodo.org/record/7595904#.Y9q7pS-B3T8)

This directory should be untarred and placed into a place accessible to the BatAnalysis code. If the user is planning on having a single directory hold all the downloaded BAT survey data then the pattern map folder should be renamed to: "noise_pattern_maps" and placed within the "main" survey observation ID directory. In other words the directory structure should look like this:

    - /path/to/main/dir/
        -0000123456/
        -0000153493/
        -noise_pattern_maps/

This will allow the BatAnalysis code to easily find the directory making calls to analyze data simpler.
Alternatively, the noise pattern map directory can be placed anywhere accessible by python and the path to this directory (including the name of the directory) can simply be passed to relevant calls to the BatAanlysis code.

### 2. With the package now installed on your local machine we can now import the package alongside other packages that we will need to analyze BAT survey data
#### *BatAnalysis requires [pyXspec](https://heasarc.gsfc.nasa.gov/xanadu/xspec/python/html/index.html) and [Heasoftpy](https://heasarc.gsfc.nasa.gov/docs/software/lheasoft/heasoftpy.html) which can be set up as a part of the installation of the heasoft tools from source*. More information for installing heasoft from source can be found at: [https://heasarc.gsfc.nasa.gov/lheasoft/download.html](https://heasarc.gsfc.nasa.gov/lheasoft/download.html). 

#### *Caldb will also need to be installed. There is a way to install it in which it retrives the most up-to-date calibration files directly from HEASARC (but this will prevent you from conducting analyses if you are not connected to the internet).*

In [15]:
import batanalysis as ba
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import astropy.units as u

Run the next line for interactive plots:

In [16]:
%matplotlib notebook

This next line is for dealing with heasoftpy outputs in jupyter notebooks

In [17]:
import os
os.environ['HEADASNOQUERY']='1'

In [18]:
# Also, one of the developers needs to work behind a proxy server
try:
    from dmptools import proxyfix
    proxyfix()
except:
    pass

You will also want a place to locally store copies of the downloaded data.  (Storing them within the same directory as your code is extremely messy and breaks version control).  The following makes a subdirectory in /tmp if you haven't already made one, but it is recommended to use a persistent data directory on a big disk so you don't have to download the data so often.
```
ba.datadir('/local/bigdisk/batdata', mkdir=True, persistent=True)
```
will set you up and will be available even for later python runs

In [19]:
# Set up data directory in /tmp if necessary (don't pollute current working directory)
# If you don't want to download the same files repeatedly when /tmp runs out, put them somewhere else
if ba.datadir().resolve() == Path('.').resolve():
    newdir = Path("/tmp/batdata")
    ba.datadir(newdir, mkdir=True)
    print(f"Setting data directory to {ba.datadir()}")
print(f"Data directory is {ba.datadir()}")

Data directory is /private/tmp/batdata


### 3. Now that we have loaded everything, we need survey data to analyze. This can be easily downloaded via the BatAnalysis code and the astroquery package.

#### *More information on the astroquery interface can be found at: [https://astroquery.readthedocs.io/en/latest/](https://astroquery.readthedocs.io/en/latest/)*

The batanalysis utility `ba.from_heasarc()` encapsulates this.

Below we will show an example of a simple query from heasarc for observations named `FRB180916.J0158+65` and how we can easily download this data.  This does not find all the data with the source in the BAT FOV, because any given point in the sky is in the BAT FOV about ~1/8 of the time. 

In [20]:
object_name='FRB180916.J0158+65'

table = ba.from_heasarc(object_name)

Lets print the table to see what results we got from our query

In [21]:
print(f"{len(table)} rows found")
table.pprint_all()

280 rows found
                                      NAME                                          OBSID       RA      DEC       START_TIME    PROCESSING_DATE XRT_EXPOSURE UVOT_EXPOSURE BAT_EXPOSURE ARCHIVE_DATE        SEARCH_OFFSET_       
                                                                                               deg      deg          mjd              mjd            s             s            s           mjd                                  
-------------------------------------------------------------------------------- ----------- -------- -------- ---------------- --------------- ------------ ------------- ------------ ------------ ----------------------------
FRB180916.J0158+65                                                               00013201172 29.52796 65.72303 59390.2483217593           59400   4107.58400    4090.55400   4129.00000        59401 0.719 (FRB180916.J0158+65)\n
FRB180916.J0158+65                                                               

There are quite alot of entries. To speed things up we are only going to download the first two datasets of survey data that has observations of FRB180916. This can be done by passing in the first two elements of the table to the BatAnalysis download_swiftdata function. We also manually add an observation that will stay stationary for us to analyze since the queried table changes as more observations are taken.

This is a wrapper around the swifttools.swift_too.Data class, which knows how to extract data from the heasarc archives, or from quicklook.


In [32]:
from pathlib import Path
print(table[:2])
result = ba.download_swiftdata([*table["OBSID"][:2],"00013201172"])
# What directories did we get?
print(result)
sorted(ba.datadir().glob('*'))

                                      NAME                                       ...
                                                                                 ...
-------------------------------------------------------------------------------- ...
FRB180916.J0158+65                                                               ...
FRB180916.J0158+65                                                               ...


/Users/tparsota/miniforge3/lib/python3.12/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: SWIFT_TLE_ARCHIVE.txt.21021.84516588.gz exists and not overwritten (set clobber=True to override this).
/Users/tparsota/miniforge3/lib/python3.12/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00013201122pat.fits.gz exists and not overwritten (set clobber=True to override this).
/Users/tparsota/miniforge3/lib/python3.12/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00013201122pjb.par.gz exists and not overwritten (set clobber=True to override this).
/Users/tparsota/miniforge3/lib/python3.12/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: SWIFT_TLE_ARCHIVE.txt.21182.87130952.gz exists and not overwritten (set clobber=True to override this).
/Users/tparsota/miniforge3/lib/python3.12/site-packages/swifttools/swift_too/swift_data.py:394: UserWarning: sw00013201122pob.cat.gz exists and not overwritten (set clobber=True t

{'00013201172': {'obsid': '00013201172', 'success': True, 'obsoutdir': PosixPath('/private/tmp/batdata/00013201172'), 'quicklook': False, 'data': Swift_Data(username='anonymous',obsid='00013201172',quicklook='False',auxil='True',bat='True',xrt='False',uvot='False',log='False',tdrss='True')}, '00013201122': {'obsid': '00013201122', 'success': True, 'obsoutdir': PosixPath('/private/tmp/batdata/00013201122'), 'quicklook': False, 'data': Swift_Data(username='anonymous',obsid='00013201122',quicklook='False',auxil='True',bat='True',xrt='False',uvot='False',log='False',tdrss='True')}, '00013201002': {'obsid': '00013201002', 'success': True, 'obsoutdir': PosixPath('/private/tmp/batdata/00013201002'), 'quicklook': False, 'data': Swift_Data(username='anonymous',obsid='00013201002',quicklook='False',auxil='True',bat='True',xrt='False',uvot='False',log='False',tdrss='True')}}


[PosixPath('/private/tmp/batdata/00013201002'),
 PosixPath('/private/tmp/batdata/00013201067'),
 PosixPath('/private/tmp/batdata/00013201082'),
 PosixPath('/private/tmp/batdata/00013201122'),
 PosixPath('/private/tmp/batdata/00013201172'),
 PosixPath('/private/tmp/batdata/tdrss'),
 PosixPath('/private/tmp/batdata/trend')]

By default, the download_swiftdata function downloads the observation ID folders in the current working directory.  It skips the XRT and UVOT data directories, which are much larger than the BAT data.  It also skips downloads if the observation directory exists. This behavior can be changed. To see the options available to this function we can run:

In [23]:
ba.download_swiftdata?

Signature:
ba.download_swiftdata(
    observations,
    reload=False,
    fetch=True,
    jobs=10,
    bat=True,
    auxil=True,
    log=False,
    uvot=False,
    xrt=False,
    tdrss=True,
    save_dir=None,
    **kwargs,
) -> dict
Docstring:
Download Swift data from HEASARC or quicklook sites to a local mirror directory.

If the data already exists in the mirror, it is not reloaded unless it is from
a quicklook site, or if reload is set.

Data for observations can be selected by instrument or by filename match.

Observations are specified as a list of OBSIDs, or a table with an 'OBSID' field.

Match is a string or list of strings that match the filenames using unix globbing rules.
e.g. `match=['*brtms*', '*sao.*']` will match both the BAT 64 ms rates and the
instrument auxiliary orbit information file (if bat=True and auxil=True are set) for
each observation.

The result is returned in a dict indexed by OBSID.  The 'data' element of an OBSID's
dict entry is a `swifttools.swift_too.S

This shows the docstring of the function which shows that there is a save_dir parameter that can be passed to direct the function to save the data to the specified folder. The same syntax of placing a '?' after a function or class name will allow us to get information on any functions or classes that are used here in this notebook. Including those that the BatAnalysis code uses. For example to see the default values that are passed to the heasoft batsurvey script we can do:

In [24]:
import heasoftpy as hsp
hsp.swift.batsurvey?

Signature: hsp.swift.batsurvey(args=None, **kwargs)
Docstring:
Parameters
----------
indir        (Req) :  Name of input observation directory  (default: ) 
outdir       (Req) :  Name of output results directory  (default: ) 
incatalog          :  Name of input analysis catalog (or NONE)  (default: NONE) 
ncleaniter         :  Number of clean iterations  (default: 2) 
energybins         :  Energy bins for survey results (keV)  (default: 14-20,20-24,24-35,35-50,50-75,75-100,100-150,150-195) 
elimits            :  Energy limits for DPI quality checking (keV)  (default: 14-195) 
timesep            :  Specify time binning method (SNAPSHOT or DPH)  (default: SNAPSHOT) 
keepbits           :  Number of significant sky image bits to keep (or ALL)  (default: 7) 
keep_sky_images       :  Keep sky images? (ALL, NONE or LAST)  (default: LAST) 
poivarmap          :  Make poisson variance maps?  (default: yes) 
pointing_check       :  Check for bad attitude data during pointing?  (default: yes) 
fil

Thus, we can see important information that is used in the heasoft batsurvey call. 

If we wanted to do a more thorough query of the observation IDs that have observations of the source of interest, we need to query heasarc for the location of the source instead of the name. This can be easily done by using the BatAnalysis package and the [SwiftBat](https://github.com/lanl/swiftbat_python/) packages. 

To do so, we would do the following:

In [25]:
import swiftbat

#first create a source object with the swiftbat backage
object_location = swiftbat.simbadlocation("FRB180916")

object_batsource = swiftbat.source(ra=object_location[0]*u.deg, dec=object_location[0]*u.deg,\
                                   name="FRB180916")

#then query the database for all the observation IDs that occured within a time 
# frame of interest
queryargs = dict(time="2004-12-15..2006-10-27", fields='All', resultmax=0)
table_everything = ba.from_heasarc(**queryargs)

#set a minimum exposure time for each observation to have 
#considering the partial coding fraction 
minexposure = 1000     # cm^2 after cos adjust

#calculate the exposures of each observation ID from our heasarc query
exposures = u.Quantity([object_batsource.exposure(ra=row['RA'], dec=row['DEC'],roll=row['ROLL_ANGLE'])[0] for row in table_everything])

#filter the ones that meet our minimum exposure condition
table_exposed = table_everything[exposures.value > minexposure]

print(f"Finding everything finds {len(table_everything)} observations, of \
        which {len(table_exposed)} have more than {minexposure:0} cm^2 coded")

#then pass table_exposed into download_swiftdata to download the results
# result = ba.download_swiftdata(table_exposed)

#after filter the downloaded data based on which were downloaded properly
# final_obs_ids=[i for i in table_exposed['OBSID'] if ba.datadir().joinpath(i).exists()]

Finding everything finds 11989 observations, of         which 1569 have more than 1000 cm^2 coded


We wont be using this to download data of this source but this code will work for any source of interest and will ensure that you conduct a more thorough search of BAT data.

Now that we have our data, we can proceed with analyzing it.

In [26]:
object_batsource.exposure?

Signature: object_batsource.exposure(ra, dec, roll)
Docstring: <no docstring>
File:      ~/Library/CloudStorage/OneDrive-NASA/BAT_SCRIPTS/SWIFT_BAT_CODE/swiftbat_python/swiftbat/swinfo.py
Type:      method

### 4. Analyzing the data

In order to analyze the survey data, we need to create a BatSurvey object with the observation ID. There are a number of things that we can pass to the object as well. These can be viewed by doing:

In [27]:
ba.BatSurvey?

Init signature:
ba.BatSurvey(
    obs_id,
    obs_dir=None,
    input_dict=None,
    recalc=False,
    verbose=False,
    load_dir=None,
    patt_noise_dir=None,
)
Docstring:     
A general Bat Survey object that holds all information necessary to analyze Bat survey data.

Attributes
---------------
obs_id : str
    observation ID
obs_dir : str or None
    Directory that the observation ID folder resides within
survey_input : dictionary
    Dictionary that holds the information that was passed to heasoft's batsurvey
result_dir : str
    The directory that holds the output of the heasoft batsurvey calculations
batsurvey_result : heasoftpy Result object
    The output of calling heasoftpy batsurvey
pointing_flux_files : list of strings
    A list of the pointing files created by heasoftpy bat_survey for the specified obs_id
pointing_ids : list of strings
    The pointing  ids for the successfully analyzed pointings associated with the analyzed obs_id
pointing_info : dictionary of diction

One of the more important parameters that can be passed to the BatSurvey object is the input_dict parameter. This dictionary gets passed to heasoftpy's batsurvey code which does the bulk of the computation. Another important parameter is the load_file parameter which allows one to load progress that they have saved at an earlier point in time (which we will show in a few cells).

Lets go ahead and create our batsurvey object for observation ID 00013201172, which we can get by looking at the astroquery table or the name of the downloaded file. (As a reminder, we manually specified this since the Heasarc query returns new entries as more observations are taken.)

In [34]:
obs=ba.BatSurvey(obs_id='00013201172')

We did not pass a value in for the obs_dir parameter which means that the BatAnalysis package will automatically use the directory specified by the ba.datadir() function.

We also did not pass a dictionary in for the input_dict parameter which means that we are using the default bat survey catalog that is included in the BatAnalysis package. There are other defaults that are set and can be found in the documentation. If we wanted to change any of these, we would create a dictionary with key value pairs that match up with what heasoft's batsurvey expects.

Next, lets define the name of our source, ***as it is listed in our batsurvey catalog***. This name will be used later in the anlysis to specify that we want to do spectral fittings/sourcd detction for this FRB. 

In [ ]:
source_name="FRB180916" #This is the name of the source that we are interested in 

The default batsurvey catalog file includes FRB180916 in it but if it didn't we would have to create a custom catalog with the source of interest. Lets do this for another source 1ES1927+654 with RA=291.831417, Dec=65.565056, galactic longitude=96.984022, and galactic latitude=20.960860.

In [ ]:
custom_catalog_file=ba.create_custom_catalog("1ES1927+654", 291.831417, 65.565056, 96.984022, 20.960860)
!ls

The output catalog is named custom_catalog.cat by default. This function returns the path to the custom catalog that was just created. 

While we have added a single source here, the create_custom_catalog function can also take a list of source names, and lists of RA, Dec, galactic latitude and galactic longitudes in the same order. The custom catalog includes the original sources in the bat survey catalog and the added source(s) of interest. This is important so that the batsurvey code can clean the brightest known sources from teh detector plane histograms.

We wont be using this custom catalog in our creation of a BatSurvey object (which calls heasoft's batsurvey) but if we wanted to use it we would first create a dictionary and pass that into BatSurvey. This could be implemented as:
```
batsurvey_input=dict(incatalog=f"{custom_catalog_file}", detthresh="10000")
obs=ba.BatSurvey('00013201002', input_dict=batsurvey_input)
```
If we also wanted to use the finest time resolution of the survey data (the detector plane histograms, or DPH, instead of the snapshots, which are the larger time resolution analysis of the survey data) we would do something like:
```
batsurvey_input=dict(incatalog=f"{custom_catalog_file}", detthresh="10000",timesep="DPH")
obs=ba.BatSurvey('00013201002', input_dict=batsurvey_input)
```
Since we wont be using the custom catalog we will remove it.

In [ ]:
!rm custom_catalog.cat

Since running batsurvey can also take a while. We may not run this operation again in the future. As a result we can save out progress and resume our analysis at a later time without reruning things. To save your progress we simply do:

In [ ]:
obs.save()

We can now delete our obs object and reload it, simulating stopping our work.

In [ ]:
del(obs)

To resuming where we last stopped, we simply load the saved file as:

In [ ]:
obs=ba.BatSurvey('00013201002')

which will search the datadir() for a directory that was created in a previous BatSurvey initialized call. If the result directory lives elsewhere, the user can specify this with the ```load_dir``` parameter.

Lets continue our analysis now. We now need to merge the pointings based on the source name. To do this we simply do:

In [ ]:
obs.merge_pointings()

With the pointings merged (there was only one pointing for this observation ID but others may have many more) we can now generate the PHA files for the source(s) of interest. The names of the sources are changed with respect to how they may be shown in the catalog file. Spaces and slashes are replaced with underscores. So
```Crab Nebula/Pulsar``` in the catalog would become ```Crab_Nebula_Pulsar```.

If we wanted to generate Pulse Height Amplitude (PHA) files for all the sources in our custom catalog (which includes all of the original sources in the bat survey catalog then the user can pass ```None```.

For our analysis here, we can simply do:

In [ ]:
obs.calculate_pha("FRB180916")

This function can also be used to calculate upper limits on the flux of an object by using the N times background counts as the spectrum itself. Where N is the significance of the upper limit the user would like to obtain. 

With the PHA file created we can analyze the spectrum of the object for this pointing. We first need to calcualate the detector response matrix (DRM) for the PHA file. To do this we use the ```calc_response``` function and pass in a single PHA file of a list of PHA files. We can access the list of PHA files associated with the survey observation by using the ```get_pha_filenames``` method. 

In [ ]:
pha_files=obs.get_pha_filenames()
print(pha_files)

As we can see above, there is one PHA file for the single source that we had specified and for the single pointing belonging to this survey data observation ID. To calculate the DRM we simply do:

In [ ]:
ba.calc_response(pha_files)

Now that we have calculated the DRM for the PHA file we can fit it with a very simple spectrum using our ```fit_spectrum``` function. By default, this function fits a simple powerlaw to the spectrum and plots it. The user can also specify a xspec model that includes a cflux component by passing in a string to the ```generic_model``` argument. In this case the user can also use the ```setPars``` argument to set the default values of the model components. See the equivalent ```setPar``` used in pyXspec on how the syntax should be formatted [https://heasarc.gsfc.nasa.gov/docs/xanadu/xspec/python/html/model.html](https://heasarc.gsfc.nasa.gov/docs/xanadu/xspec/python/html/model.html). 

This function takes a single string of the PHA file of interest which we pass by accessing the first element of our ```pha_files``` list. By default, the function will use a ```cflux*powerlaw``` spectrum to fit the data from 14-195 keV (the enrgy range of the BAT survey data), it will use the cstat fit statistic, calculate the $1\sigma$ confidence interval, and it will plot and save the spectrum in the same directory as the PHA file. The resulting spectral model with its values, lower/upper limits, and any fit error codes, are saved to the BatSurvey object for easy accesss by the user later.  

In [ ]:
ba.fit_spectrum(pha_files[0], obs)

To access the spectral information, we simply access the dictionary of the pointing ID of interest and the sub-dictionary for the source of interest. This would look like getting the pointing ID of interest and then using it to access information related to that pointing ID for the source of interest.

Here we only have one pointing ID which we can get by doing:

In [ ]:
pointing_ids=obs.get_pointing_ids()
print(pointing_ids)

Now, we can supply this pointing ID to get the dictionary associated with it by doing:

In [ ]:
pointing_dict=obs.get_pointing_info(pointing_ids[0])
print(pointing_dict)

We see that there is key within this dictionary that is the source of interest FRB180916 which contains the pyXspec spectral information that we want. WE can either access this from the ```pointing_dict``` dictionary or obtain the source dictionary directly as:

In [ ]:
source_dict=obs.get_pointing_info(pointing_ids[0], source_id=source_name)
print(source_dict)

Where the flux, for example, from our ```cflux*po``` xspec model can be accessed as:

In [ ]:
print(source_dict["model_params"]["lg10Flux"])

We can see above that the ```source_dict``` dictionary has the spectral fit values, errors, and error codes that are produced by pyXspec in its fitting. We also save the full Xspec .xcm file which allows the user to easily reload the pyXspec analysis of this given PHA file and elaborate on it.

If there are any `T`'s in the error code (as we see here) that is an indicator that the fit is poor, which is also clear from the plotted spectrum and model fit. 

We have also created a convenience function, called ```calculate_detection```, to determine if there was a detection or not based on the flux of the object (which is why it is important to include a cflux component in the generic_model imput). If there was not a detection, then the function automatically calculates flux upper limits for the source. This function takes the BatSurvey object the source(s) of interest (which may get passed to ```calculate_pha```). There are also other parameters that can be passed to plot the upper limit fit for example.

In [ ]:
ba.calculate_detection(obs,source_name)

Here, we find no detection of FRB180916 and automatically calculate the 5$\sigma$ upper limit. The information for this flux upperlimit can be obtained from the BatSurvey object by doing:

In [ ]:
source_dict=obs.get_pointing_info(pointing_ids[0], source_id=source_name)
print(source_dict)
print('The upper limit is',source_dict["nsigma_lg10flux_upperlim"])

### 5. Analyzing sets of survey data

When we queried the heasarc database, we acquired a large number of observation IDs that contained observations of the source of interest, which is FRB180916 in our case. We only downloaded 2 out of the many but in principle we could have downloaded all these datasets and analyzed them in a streamlined pipeline that allows us to acquire spectra, light curves, and fluxes for all these observations easily. 

It is important to note that some of the data sets that appeared with our query may not actually contain survey data and, as a result, do not actually contain information on the source that we are intersted in. This caveat can be easily handled in the BatAnaysis code, where we can determine if there was an error thrown with the heasoft batsurvey call that is a result of this occurence. This error in the database can easily be handled in python by exception handling. 

With this caveat, we can show the general framework for analyzing batches of survey data. 

After downloading the data, we can get the observation IDs from the astroquery table. We use these observation IDs to create BatSurvey objects that allow us to easily analyze each survey dataset. For our simple analysis, we only downloaded two observation IDs so our for loop will be slightly modified.

In [ ]:
source_name="FRB180916"

#create an empty list to hold our BatSurvey objects
batsurvey_obs=[]

#loop over the observation IDs
for i in table["OBSID"][:2]:
    print(i)
    try:
        #try to create the BatSurvey object
        obs=ba.BatSurvey(i,recalc=True)
        #if there is actually survey data present, save our progress so we dont have to run heasoft batsurvey again
        obs.save()
        #append it to our list of BatSurvey objects
        batsurvey_obs.append(obs)
    except ValueError:
        #If there is an exception thrown, it is due to there being an error with batsurvey, which is mostly due to 
        # there actually being no survey data in the specified observation ID
        print("Obsid has no survey data")

Here we proactively saved our BatSurvey objects after heasoft batsurvey was run. This allows us to easily restart from this point without rerunning heasoft's batsurvey which can take a while at times. The code to do this is:
```
batsurvey_obs=[]
for i in table["OBSID"][:2]:

    try:
        batsurvey_obs.append(ba.BatSurvey(i, load_file=os.path.join("/Path/to/results/dir/,"%s_surveyresult/batsurvey.pickle"%(i))))
    except FileNotFoundError:
        pass

```
This code will go through each of the same queried heasarc observation IDs and load the valid survey data objects.

Once we have our list of BatSurvey objects we can now loop over each observation and create PHA files, DRMs, fit spectra, and obtain fluxes. This would be done as follows:

In [ ]:
for obs in batsurvey_obs:

    print("Running calculations for observation id", obs.obs_id)
    obs.merge_pointings()
    obs.calculate_pha(id_list=source_name, clean_dir=True)
    pha_list=obs.get_pha_filenames(id_list=source_name)
    ba.calc_response(pha_list)

    #Loop over individual PHA pointings
    for pha in pha_list: 
        ba.fit_spectrum(pha, obs)
        
    ba.calculate_detection(obs, source_name)


With all of the observation IDs analyzed we can summarize the results in a nice table. This is simply done by calling the convience function `print_parameters` and passing the list of BatSurvey observations. Optionally, the user can supply key values within the pointing dictionaries that the user would like printed, values that specify if the outputted table is formatted in $\LaTeX$, and if the table should be saved to a file.

In [ ]:
ba.print_parameters(batsurvey_obs,source_name, values=["met_time","utc_time", "exposure","lg10Flux", "PhoIndex"])

Another important analysis that we can conduct is looking at the light curve of the source across all the downloaded survey data. This is done by first merging all of the pointing ID counts into a single file with:

In [ ]:
ba.combine_survey_lc(batsurvey_obs)

The output file is in the base directory which contains all the \*_surveyresult directories.

We can then plot the resulting light curve in units of MET, UTC or MJD.

In [ ]:
fig, axes=ba.plot_survey_lc(batsurvey_obs, id_list=source_name, time_unit="UTC", values=["rate","snr", "flux", "PhoIndex", "exposure"], calc_lc=True)

### 6. Parallel Data Analysis

The foor loops shown above can be run in a parallel manner by calling the parallel submodule of BatAnalysis. For example, to loop through a set of observation IDs and create our list of BatSurvey objects with 12 processes we can do:

```
batsurvey_obs=ba.parallel.batsurvey_analysis(table_exposed['OBSID'], input_dict=input_dict, nprocs=12)
```

Then, if we want to analyze these observation IDs by fitting spectra and determining if there was a 3$\sigma$ detection using 2 processes, we would do:

```
ba.parallel.batspectrum_analysis(batsurvey_obs, source_name, nprocs=2)
```


### 7. Mosaic Data Analysis

The mosaicing analysis is very simple to conduct. This step involves identifying the individual survey data that will be included in the creation of the mosaic images and then specifying the time binning for the creation of the resulting mosaic images.

To compile the list of survey observations that will be included in the mosaic images we do:

In [ ]:
outventory_file=ba.merge_outventory(batsurvey_obs)

To define the bin sizes of the mosaic images we use the ```group_outventory``` function. By default the start and end times of the time binnings are set to be the earliest and latest times of the list of surveys that were compiled into the outventory file in the previous cell.

To define monthly time bins of mosaicing data from the start of the BAT mission to the 22 month operational month of BAT we would do:

In [ ]:
time_bins=ba.group_outventory(outventory_file, np.timedelta64(1, "M"), start_datetime=Time("2004-12-24"), end_datetime=Time("2006-10-27"))

Finally, to do the calculations for the creation of the mosaic images we would simply do:


In [ ]:
mosaic_list, total_mosaic=ba.parallel.batmosaic_analysis(batsurvey_obs, outventory_file, time_bins, nprocs=1)

As a note, the above cell can take up  to ~10 GB of memory per each proc that is used to parallelize the calculation so the user may need to set nproc to an appropriate value such that they do not use all the RAM on their system.

To detect sources in each mosaic image and fit their spectra we would do:

In [ ]:
mosaic_list=ba.parallel.batspectrum_analysis(mosaic_list, source_name, nprocs=1)

and for the total "time-integrated" mosaiced image we would do:

In [ ]:
total_mosaic=ba.parallel.batspectrum_analysis(total_mosaic, source_name, nprocs=1)